parameter tune
- tour terbanyak
- day
- time 


experiment
- tour terbanyak / epoch
- day / epoch
- time / epoch

In [2]:
import numpy as np
from pprint import pprint
from copy import deepcopy
from utils.base_model import BaseModel
from mealpy.custom_based.CustomBF import BF

In [12]:
## Setting parameters
base_params = {
    "model": BF,
    "maut_weights" : {
        "popular" : 1,
        "cost" : 1,
        "time" : 1
    },
    "time_start": 8,
    "time_end": 20,
    
    "n_days": 50,
    "hotel_id": 5,
    # "tour_ids": [0, 2, 4, 6, 7, 8, 9, 20, 23, 25, 29, 30, 31, 32, 33, 35, 40, 55, 42, 56],
    "tour_ids": list(range(0, 100)),
    
    "minmax": "max",
    "log_to": "console",
}
model_params = {
    "epoch": 1,           # iteration
}
model = BaseModel(base_params=base_params, model_params=model_params)
model.train()


2022/12/19 06:48:23 PM, INFO, mealpy.custom_based.CustomBF.BF: Solving single objective optimization problem.
2022/12/19 06:55:40 PM, INFO, mealpy.custom_based.CustomBF.BF: >Problem: P, Epoch: 1, Current best: 0.035241129267299685, Global best: 0.035241129267299685, Runtime: 56.54245 seconds



Day-1 Current Best solution: [1, 0, 6, 9, 3, 2, 4, 8] - [1, 0, 6, 9, 3, 2, 4, 8], Obj = Total Distance: 0.035241129267299685, pop_size: 3628800

Final Best solution: [[[Place id: 1 -> Name: Tourism Zone Malioboro, Type: Tourist Attraction, Rating: 4.7, Cost: 0, Spend Time (s): 3600, Place id: 0 -> Name: MALIOBORO JOGJAKARTA, Type: Tourist Attraction, Rating: 4.8, Cost: 0, Spend Time (s): 5400, Place id: 6 -> Name: Titik Nol Km Yogyakarta, Type: Tourist Attraction, Rating: 4.8, Cost: 0, Spend Time (s): 3600, Place id: 9 -> Name: Yogyakarta Northern Square, Type: Tourist Attraction, Rating: 4.6, Cost: 0, Spend Time (s): 3600, Place id: 3 -> Name: Bringin Kembar Yogyakarta, Type: Tourist Attraction, Rating: 4.6, Cost: 0, Spend Time (s): 5400, Place id: 2 -> Name: Plengkung Gading, Type: Tourist Attraction, Rating: 4.6, Cost: 0, Spend Time (s): 1800, Place id: 4 -> Name: Rumah Hantu Malioboro, Type: Tourist Attraction, Rating: 4.5, Cost: 25000, Spend Time (s): 2700, Place id: 8 -> Name: T

In [13]:
print(f"Count Node: {len(np.hstack([x[0] for x in model.best_solution]))}")
for idx, day in enumerate(model.best_solution):
    print("\nDay", idx+1)
    total_time = 8*3600
    tour_nodes = day[0]
    for i in range(len(tour_nodes)):
        time = 0
        spend_time = 0
        if i == 0:
            time = model.conn.get_hotel_dist_matrix(origin_id=model.hotel_node.id, dest_id=tour_nodes[i].id, hotel2tour=True)
            spend_time = 0
        elif i+1 == len(tour_nodes):
            spend_time = tour_nodes[i].spend_time
            time = model.conn.get_hotel_dist_matrix(origin_id=model.hotel_node.id, dest_id=tour_nodes[i].id, hotel2tour=False)+spend_time
            print("To Hotels -", 20*3600, int(np.floor((total_time+time)/3600)))
        else:
            spend_time = tour_nodes[i].spend_time
            time = model.conn.get_tour_dist_matrix(origin_id=tour_nodes[i].id, dest_id=tour_nodes[i+1].id)+spend_time
        total_time += time
        
        jam_berangkat = f"{int(np.floor((total_time-(time-spend_time))/3600))}:{int(np.round(((total_time-(time-spend_time))%3600)/60))}"
        jam_sampai = f"{int(np.floor(total_time/3600))}:{int(np.round((total_time%3600)/60))}"
        print(tour_nodes[i].id, tour_nodes[i].rating, tour_nodes[i].tarif, time, (time-spend_time), tour_nodes[i].jam_buka, tour_nodes[i].jam_tutup, total_time, jam_berangkat, jam_sampai)

    print(np.round(total_time/3600))
    
if len(model.outlier_solution) != 0:
    print("\nOutlier")
    pprint(model.outlier_solution)


Day 1
1 4.7 0 119.0 119.0 [0, 1] [24, 0] 28919.0 8:0 8:2
0 4.8 0 5622.0 222.0 [0, 1] [24, 0] 34541.0 9:32 9:36
6 4.8 0 3687.0 87.0 [0, 1] [24, 0] 38228.0 10:36 10:37
9 4.6 0 3963.0 363.0 [0, 1] [24, 0] 42191.0 11:37 11:43
3 4.6 0 5449.0 49.0 [0, 1] [24, 0] 47640.0 13:13 13:14
2 4.6 0 2502.0 702.0 [0, 1] [24, 0] 50142.0 13:44 13:56
4 4.5 25000 2948.0 248.0 [16, 0] [18, 0] 53090.0 14:41 14:45
To Hotels - 72000 15
8 4.7 0 2512.0 712.0 [0, 1] [24, 0] 55602.0 15:15 15:27
15.0

Outlier
[[[Place id: 5 -> Name: Fort Vredeburg Museum, Type: Tourist Attraction, Rating: 4.6, Cost: 3000, Spend Time (s): 7200,
   Place id: 7 -> Name: Museum Sonobudoyo Unit I, Type: Tourist Attraction, Rating: 4.7, Cost: 3000, Spend Time (s): 7200],
  0]]
